<a href="https://colab.research.google.com/github/Miguel1897/01-Tarea-Repaso-ReactJs/blob/master/DeteccionEnfermedades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importación de Librerías



In [1]:
# librerías necesarias
!pip install -q kaggle
!pip install torch torchvision

# Importa las librerías
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt


Configuración de la API de Kaggle y Descarga del Dataset


In [2]:
# Configura el archivo de autenticación kaggle.json
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Descarga el dataset desde Kaggle
!kaggle datasets download -d kanishk3813/pathogen-dataset --unzip

# Verifica la ubicación del dataset descargado
!ls /content

# Ajusta las rutas para cargar el dataset en PyTorch
from torchvision import datasets, transforms
import os

# Define la ruta del directorio según el nombre de la carpeta descomprimida
dataset_dir = '/content/pathogen-dataset'  # Cambia esto si el nombre de la carpeta es diferente

# Transforma las imágenes
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])



cp: cannot stat '/content/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/kanishk3813/pathogen-dataset
License(s): apache-2.0
 99% 1.42G/1.43G [00:20<00:00, 71.0MB/s]
100% 1.43G/1.43G [00:20<00:00, 73.8MB/s]
pathogen  sample_data


Preparación del Dataset

In [3]:
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import os

# Especifica la ruta del dataset descargado
dataset_dir = '/content/pathogen'  # Ruta donde están las carpetas de clases

# Define las transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Redimensionar las imágenes a 128x128
    transforms.ToTensor()  # Convertir las imágenes a tensores
])

# Cargar el dataset completo con todas las clases
dataset = datasets.ImageFolder(dataset_dir, transform=transform)

# Verificar las clases detectadas en el dataset
print("Clases:", dataset.classes)

# Dividir el dataset en entrenamiento y validación (80% entrenamiento, 20% validación)
train_size = int(0.8 * len(dataset))  # 80% para entrenamiento
valid_size = len(dataset) - train_size  # El resto para validación

# Dividir el dataset en dos
train_data, valid_data = random_split(dataset, [train_size, valid_size])

# Crear los DataLoaders para entrenamiento y validación
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32)

# Verificar la cantidad de muestras en los DataLoaders
print(f'Tamaño del conjunto de entrenamiento: {len(train_data)}')
print(f'Tamaño del conjunto de validación: {len(valid_data)}')

# Verificar algunas clases
print(f'Primeras clases de entrenamiento: {train_data.indices[:5]}')


Clases: ['Bacteria', 'Fungi', 'Healthy', 'Pests', 'Virus']
Tamaño del conjunto de entrenamiento: 31997
Tamaño del conjunto de validación: 8000
Primeras clases de entrenamiento: [37797, 34465, 32332, 6109, 8686]


Definición del Modelo CNN

In [4]:
import torch
import torch.nn as nn

class PlagaDetectorCNN(nn.Module):
    def __init__(self):
        super(PlagaDetectorCNN, self).__init__()

        # Capas convolucionales
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Conv1: 3 canales de entrada, 32 canales de salida
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # Conv2: 32 canales de entrada, 64 de salida
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # Conv3: 64 canales de entrada, 128 de salida

        # Capa de max-pooling
        self.pool = nn.MaxPool2d(2, 2)  # Tamaño del kernel 2x2

        # Capas completamente conectadas
        self.fc1 = nn.Linear(128 * 16 * 16, 512)  # Capa densa (128x16x16 es la dimensión de salida de las convoluciones)
        self.fc2 = nn.Linear(512, len(train_data.classes))  # La salida final tiene tantas clases como train_data.classes

    def forward(self, x):
        # Aplicamos las convoluciones seguidas de ReLU y MaxPooling
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))

        # Aplanamos la salida para pasarla a la capa totalmente conectada
        x = x.view(-1, 128 * 16 * 16)  # Aplanar: las dimensiones deben coincidir con la capa fc1

        # Capas completamente conectadas
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        return x


Configuración del Dispositivo (CPU o GPU)

In [6]:
# Suponiendo que train_data_full es el conjunto de datos completo ImageFolder
train_data_full = datasets.ImageFolder(os.path.join(dataset_dir), transform=transform)

# Obtener el número de clases
num_classes = len(train_data_full.classes)

# Definir el modelo usando el número de clases
class PlagaDetectorCNN(nn.Module):
    def __init__(self, num_classes):
        super(PlagaDetectorCNN, self).__init__()

        # Capas convolucionales
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, num_classes)  # Usar num_classes aquí

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Ahora crear el modelo con el número de clases correcto
model = PlagaDetectorCNN(num_classes).to(device)


NameError: name 'device' is not defined

Definición de Funciones de Entrenamiento y Evaluación

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn

# Define el criterio de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Función para entrenar el modelo
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Mover a GPU/CPU
            optimizer.zero_grad()  # Limpiar gradientes previos
            outputs = model(images)  # Obtener las predicciones del modelo
            loss = criterion(outputs, labels)  # Calcular la pérdida
            loss.backward()  # Calcular el gradiente
            optimizer.step()  # Actualizar los parámetros
            running_loss += loss.item()  # Acumular la pérdida de la época
        # Promediar la pérdida de la época
        print(f"Epoch {epoch+1}, Pérdida promedio: {running_loss/len(train_loader)}")

# Función para evaluar el modelo en el conjunto de validación
def evaluate_model(model, valid_loader):
    model.eval()  # Modo evaluación (desactiva dropout, etc.)
    total, correct = 0, 0
    running_loss = 0.0
    with torch.no_grad():  # No calcular gradientes durante la validación
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)  # Obtener las predicciones del modelo
            loss = criterion(outputs, labels)  # Calcular la pérdida de validación
            running_loss += loss.item()  # Acumular la pérdida de validación
            _, predicted = torch.max(outputs, 1)  # Obtener las predicciones (la clase con mayor probabilidad)
            total += labels.size(0)  # Total de muestras
            correct += (predicted == labels).sum().item()  # Comparar con las etiquetas reales

    # Imprimir métricas
    print(f"Pérdida en validación: {running_loss/len(valid_loader)}")
    print(f"Precisión en validación: {100 * correct / total}%")


Entrenamiento del Modelo


In [7]:
train_model(model, train_loader, criterion, optimizer, epochs=10)
evaluate_model(model, valid_loader)



NameError: name 'train_model' is not defined

Uso del Modelo para Predicciones


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Configuración del dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Utilizando un modelo preentrenado para mejorar el rendimiento (ResNet18 en este caso)
class PlagaDetectorResNet(nn.Module):
    def __init__(self, num_classes):
        super(PlagaDetectorResNet, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Definición de transformaciones de datos con data augmentation
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

# Cargar los datos de entrenamiento y validación
dataset_dir = '/content/pathogen'  # Asegúrate de que esta ruta sea correcta
train_data = datasets.ImageFolder(dataset_dir, transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(train_data, batch_size=32)

# Inicializar el modelo, pérdida y optimizador
model = PlagaDetectorResNet(num_classes=len(train_data.classes)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Función para entrenar el modelo
def train_model(model, train_loader, criterion, optimizer, scheduler, epochs=15):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        scheduler.step()  # Actualiza el learning rate
        print(f"Epoch {epoch+1}, Pérdida promedio: {running_loss/len(train_loader):.4f}")

# Función para evaluar el modelo en el conjunto de validación usando F1-score
def evaluate_model(model, valid_loader):
    model.eval()
    total, correct = 0, 0
    running_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calcula precisión y F1-score
    accuracy = 100 * correct / total
    f1 = f1_score(all_labels, all_preds, average="weighted")
    print(f"Pérdida en validación: {running_loss/len(valid_loader):.4f}")
    print(f"Precisión en validación: {accuracy:.2f}%")
    print(f"F1-Score en validación: {f1:.4f}")

# Entrena y evalúa el modelo
train_model(model, train_loader, criterion, optimizer, scheduler, epochs=15)
evaluate_model(model, valid_loader)

# Función de predicción para una imagen individual
def predict_image(image_path, model):
    model.eval()
    transform = transforms.Compose([transforms.Resize((128, 128)), transforms.ToTensor()])
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)
    output = model(image)
    _, predicted = torch.max(output, 1)
    return train_data.classes[predicted.item()]

# Prueba con una imagen de ejemplo
image_path = '/content/pathogen/Healthy/image.jpg'  # Cambia según tu imagen de prueba
prediccion = predict_image(image_path, model)
print(f"Plaga detectada: {prediccion}")
plt.imshow(Image.open(image_path))
plt.title(f"Predicción: {prediccion}")
plt.show()

